In [2]:
# import the libraries
import pandas as pd
import os 
import shutil

### Task
Video files are under labeled-data folder contains extracted frames (png files) and a csv file which holds the coordinates of the annotated body joints. 
See below the head of the csv file of video 2. 
The task is deleting unlabeled frames (png images) within the video folders. If the video folder doesn't include any labeled frame, then the video folder with its content will be deleted.  

See below the head of the csv file of video 2

In [11]:
data = pd.read_csv(r"C://Users/gokce/documents/deeplabcut/set50fps_include_unlabeled_frames/labeled-data/2/CollectedData_ozlem.csv", header =[0,1,2])
print(data.shape)
data.head()

(10, 67)


scorer       ozlem                                      \
                   bodyparts  wing_hinge                wing_tip               
                      coords           x           y           x           y   
0  labeled-data\2\img038.png  354.774225  341.594316  676.802344  360.414141   
1  labeled-data\2\img058.png  357.696083  342.040210  677.261532  360.928264   
2  labeled-data\2\img103.png         NaN         NaN         NaN         NaN   
3  labeled-data\2\img121.png  352.445230  340.811994  673.097038  359.370620   
4  labeled-data\2\img131.png  363.119374  339.789407  673.372819  359.882012   

                                                               \
  front_tarsal_tip             front_tibia_tarsus               
                 x           y                  x           y   
0       158.334059  534.233141         219.024425  466.125063   
1       127.590739  572.898890         185.129673  499.732098   
2              NaN         NaN                NaN         NaN   
3       144.855993  476.565094         218.334766  444.815006   
4       146.551542  555.690247         202.904947  478.323708   

                     ...                                                \
  front_femur_tibia  ... hind_tarsal_tip             hind_tibia_tarsus   
                  x  ...               x           y                 x   
0        238.580210  ...      435.173482  610.248368        405.271283   
1        214.254318  ...      517.973990  582.342274        444.797598   
2               NaN  ...             NaN         NaN               NaN   
3        247.363417  ...      446.456208  629.861331        421.372919   
4        220.097512  ...      443.291545  644.339467        418.280252   

                                                                              \
              hind_femur_tibia             hind_trochanter_femur               
            y                x           y                     x           y   
0  498.328711       402.708237  385.554704            381.349524  465.863466   
1  481.077974       400.448270  382.770296            392.317559  460.381621   
2         NaN              NaN         NaN                   NaN         NaN   
3  516.021789       414.619725  407.970698            378.924276  467.784695   
4  516.503968       411.795843  410.900729            377.521107  467.407725   

                           
    hind_coxa              
            x           y  
0  366.825599  457.319981  
1  377.534450  456.685843  
2         NaN         NaN  
3  362.523664  451.384083  
4  361.773256  452.586218  

[5 rows x 67 columns]

Image103 is an unlabeled frame. Therefore, the coordinate information of the body joints are misssing. 

The unlabeled frames will be deleted by doing the following steps: 
- Iteration over video folders
- open the csv file within in the video folder
- detect the unlabeled frames from the missing rows. 
- delete the corresponding frame from the video folder. 
- if none of the frames were annotated in a video, delete video folder with all its content

In [12]:
# to check if the code below returns the unlabeled image names
data.set_index(('scorer', 'bodyparts', 'coords'), inplace=True)
unlabeled_frames_list = data.loc[data.isnull().all(axis=1),].index.tolist()
for i in unlabeled_frames_list:
    print(i.split("\\")[2])

img103.png
img161.png
img239.png


In [78]:
#save the address of labeled-data folder 
address =  r"C://Users/gokce/documents/deeplabcut/set50fps_include_unlabeled_frames/labeled-data/"

#save the names of the videos in labeled-data folder as a list.
video_list = os.listdir(address)

In [83]:
# delete unlabeled frames & videos
for i in video_list:
    address_temp = address + str(i) + "/CollectedData_ozlem.csv"
    if os.path.exists(address_temp): # if none of the frames are labeled in a video, csv file is not exist.
        data_temp = pd.read_csv(address_temp, header =[0,1,2])
        data_temp.set_index(('scorer', 'bodyparts', 'coords'), inplace=True)
        unlabeled_frames_list = data_temp.loc[data_temp.isnull().all(axis=1),].index.tolist()
        for index in unlabeled_frames_list:
            image_name = index.split("\\")[2]
            image_address = address + str(i) + "/" + image_name
            os.remove(image_address)
    else: # none of the frames of the video were annotated, the video folder will be deleted
        video_address = address + str(i) + "/"
        shutil.rmtree(video_address) # to delete a folder with all its content